In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if dir1 not in sys.path:
    sys.path.append(dir1)

os.chdir("..")

In [5]:
import numpy as np

In [1]:
# experiments_name = "explosion"
# model_type = "seq2seq"
# loss_type = "indid"

# all_ens_res_df = pd.read_csv(
#     f"results/final_results/{experiments_name}/{model_type}_{loss_type}_ln_before_all_ensembles_{experiments_name}.csv",
#     index_col=0,
# ).reset_index(drop=True)

# df_1 = pd.read_csv(
#     f"results/final_results/{experiments_name}/{model_type}_{loss_type}_ln_before_ens_num_1_{experiments_name}.csv",
#     index_col="Model name",
# )
# df_2 = pd.read_csv(
#     f"results/final_results/{experiments_name}/{model_type}_{loss_type}_ln_before_ens_num_2_{experiments_name}.csv",
#     index_col="Model name",
# )
# df_3 = pd.read_csv(
#     f"results/final_results/{experiments_name}/{model_type}_{loss_type}_ln_before_ens_num_3_{experiments_name}.csv",
#     index_col="Model name",
# )

# df_1 = df_1[df_1.columns[2:-1]]
# df_2 = df_2[df_2.columns[2:-1]]
# df_3 = df_3[df_3.columns[2:-1]]

# all_ens_res_df = pd.concat([df_1, df_2, df_3])

# all_ens_res_df

In [ ]:
metric_names = all_ens_res_df.columns[1:-1].to_list()
aggregations = [name[24:] for name in all_ens_res_df["Model name"].to_list()[:3]]

for agg in aggregations:
    print(f"Aggregation: {agg}")
    for metric in metric_names:
        mean = all_ens_res_df[all_ens_res_df["Model name"].str.endswith(agg)][
            metric
        ].mean()
        std = all_ens_res_df[all_ens_res_df["Model name"].str.endswith(agg)][
            metric
        ].std()
        print(f"{metric} = {np.round(mean, 4)} \pm {np.round(std, 4)}")
    print("-" * 30)

In [2]:
import yaml
from src.datasets.datasets import CPDDatasets
from src.ensembles.ensembles import EnsembleCPDModel
from src.metrics.evaluation_pipelines import evaluation_pipeline
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
model_type = "seq2seq"

experiments_name = "explosion"

path_to_config = "configs/" + "video" + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "indid"
args_config["num_workers"] = 2

train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

In [4]:
# train_dataset_exp, test_dataset_exp = CPDDatasets("explosion").get_dataset_()
# train_dataset_road, test_dataset_road = CPDDatasets("road_accidents").get_dataset_()

self.experiments_name: explosion
path_to_data: ../../data/explosion/
self.experiments_name: road_accidents
path_to_data: ../../data/road_accidents/


In [5]:
# for i in range(10):
#     print((train_dataset_exp[i][0] == train_dataset_road[i][0]).all())

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


In [5]:
len(test_dataset)

315

In [12]:
path_to_models_folder_exp = "saved_models/indid/explosion/ens_1"
ens_indid_exp = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_indid_exp.load_models_list(path_to_models_folder_exp)
ens_indid_exp.eval()

# path_to_models_folder_road = "saved_models/indid/road_accidents/ens_1"
# ens_indid_road = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
# ens_indid_road.load_models_list(path_to_models_folder_road)
# ens_indid_road.eval()

self.experiments_name: explosion
path_to_data: ../../data/explosion/


In [6]:
# # inputs = torch.randn(16, 16, 12288)
# # inputs = torch.ones(16, 16, 12288)

# inputs, labels = next(iter(test_dataloader))

# out_1 = ens_indid_exp.models_list[0](inputs).detach()
# out_2 = ens_indid_road.models_list[0](inputs).detach()

# (out_1 == out_2).all()

In [13]:
threshold_number = 100

threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

_ = evaluation_pipeline(
    ens_indid_exp.models_list[0],
    test_dataloader,
    threshold_list=threshold_list,
    device="cuda:0",
    model_type="seq2seq",
    margin_list=[1, 2, 4],
    # scale=args_config["predictions"]["scale"],
    verbose=True,
)

  0%|          | 0/20 [00:00<?, ?it/s]

Collectting model's outputs


100%|██████████| 20/20 [00:57<00:00,  2.89s/it]


TN: 0, FP: 315, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.9806299603174603
TN: 6, FP: 309, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:4.149206161499023, COVER: 0.8041963707588707
TN: 11, FP: 304, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:5.320634841918945, COVER: 0.7762725089064375
TN: 23, FP: 292, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:6.514285564422607, COVER: 0.7764910745119079
TN: 33, FP: 281, FN: 0, TP: 1, DELAY:0.0031746032182127237, FP_DELAY:7.355555534362793, COVER: 0.7818745220828553
TN: 39, FP: 275, FN: 0, TP: 1, DELAY:0.0031746032182127237, FP_DELAY:8.30476188659668, COVER: 0.7782378520399353
TN: 44, FP: 270, FN: 0, TP: 1, DELAY:0.0031746032182127237, FP_DELAY:8.917460441589355, COVER: 0.7789868599764433
TN: 51, FP: 262, FN: 0, TP: 2, DELAY:0.0031746032182127237, FP_DELAY:9.517459869384766, COVER: 0.7907247211562688
TN: 65, FP: 247, FN: 0, TP: 3, DELAY:0.0063492064364254475, FP_DELAY:10.088889122009277, COVER: 0.8025893996876139
TN: 76, FP: 236, FN: 0, TP: 3, DELAY:0.009523809887468815, FP_

In [14]:
model_type = "seq2seq"

experiments_name = "road_accidents"

path_to_config = "configs/" + "video" + "_" + model_type + ".yaml"

with open(path_to_config, "r") as f:
    args_config = yaml.safe_load(f.read())

args_config["experiments_name"] = experiments_name
args_config["model_type"] = model_type

args_config["loss_type"] = "indid"
args_config["num_workers"] = 2

train_dataset, test_dataset = CPDDatasets(experiments_name).get_dataset_()
test_dataloader = DataLoader(
    test_dataset, batch_size=args_config["learning"]["batch_size"], shuffle=False
)

self.experiments_name: road_accidents
path_to_data: ../../data/road_accidents/


In [15]:
len(test_dataset)

315

In [16]:
path_to_models_folder_road = "saved_models/indid/road_accidents/ens_1"
ens_indid_road = EnsembleCPDModel(args_config, n_models=10, boot_sample_size=None)
ens_indid_road.load_models_list(path_to_models_folder_road)
ens_indid_road.eval()

self.experiments_name: road_accidents
path_to_data: ../../data/road_accidents/


In [17]:
threshold_number = 100

threshold_list = np.linspace(-5, 5, threshold_number)
threshold_list = 1 / (1 + np.exp(-threshold_list))
threshold_list = [-0.001] + list(threshold_list) + [1.001]

_ = evaluation_pipeline(
    ens_indid_road.models_list[0],
    test_dataloader,
    threshold_list=threshold_list,
    device="cuda:0",
    model_type="seq2seq",
    margin_list=[1, 2, 4],
    # scale=args_config["predictions"]["scale"],
    verbose=True,
)

  0%|          | 0/20 [00:00<?, ?it/s]

Collectting model's outputs


100%|██████████| 20/20 [00:58<00:00,  2.92s/it]


TN: 0, FP: 315, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:0.0, COVER: 0.9806299603174603
TN: 6, FP: 309, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:4.149206161499023, COVER: 0.8041963707588707
TN: 11, FP: 304, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:5.320634841918945, COVER: 0.7762725089064375
TN: 23, FP: 292, FN: 0, TP: 0, DELAY:0.0, FP_DELAY:6.514285564422607, COVER: 0.7764910745119079
TN: 33, FP: 281, FN: 0, TP: 1, DELAY:0.0031746032182127237, FP_DELAY:7.355555534362793, COVER: 0.7818745220828553
TN: 39, FP: 275, FN: 0, TP: 1, DELAY:0.0031746032182127237, FP_DELAY:8.30476188659668, COVER: 0.7782378520399353
TN: 44, FP: 270, FN: 0, TP: 1, DELAY:0.0031746032182127237, FP_DELAY:8.917460441589355, COVER: 0.7789868599764433
TN: 51, FP: 262, FN: 0, TP: 2, DELAY:0.0031746032182127237, FP_DELAY:9.517459869384766, COVER: 0.7907247211562688
TN: 65, FP: 247, FN: 0, TP: 3, DELAY:0.0063492064364254475, FP_DELAY:10.088889122009277, COVER: 0.8025893996876139
TN: 76, FP: 236, FN: 0, TP: 3, DELAY:0.009523809887468815, FP_

# Parse metrics for single models

In [2]:
path_to_results = "results/bce/road_accidents/single_model_results.txt"
f = open(path_to_results, "r")
results = f.read()

results_dict = {}
metric_names = [
    "AUC",
    "Time to FA",
    "DD",
    "Max F1",
    "COVER",
    "Max COVER",
    "Max F1, m1",
    "Max F1, m2",
    "Max F1, m3",
]
model_num = 0

for i, line in enumerate(results.split("\n")):
    if model_num not in results_dict.keys():
        results_dict[model_num] = {}
    if i % 13 == 3:
        results_dict[model_num]["AUC"] = float(line.split(": ")[-1])
    elif i % 13 == 4:
        time_to_fa = float(line.split(",")[0][10:])
        delay = float(line.split(", ")[1].split(" for")[0][15:])
        results_dict[model_num]["Time to FA"] = time_to_fa
        results_dict[model_num]["DD"] = delay
    elif i % 13 == 6:
        max_f1 = float(line.split(":")[0][7:])
        results_dict[model_num]["Max F1"] = max_f1
    elif i % 13 == 7:
        cover = float(line.split(":")[0][5:])
        results_dict[model_num]["COVER"] = cover
    elif i % 13 == 8:
        max_cover = float(line.split(":")[0][9:])
        results_dict[model_num]["Max COVER"] = max_cover
    elif i % 13 == 9:
        max_f1_m1 = float(line.split(" for")[0][23:])
        results_dict[model_num]["Max F1, m1"] = max_f1_m1
    elif i % 13 == 10:
        max_f1_m2 = float(line.split(" for")[0][23:])
        results_dict[model_num]["Max F1, m2"] = max_f1_m2
    elif i % 13 == 11:
        max_f1_m3 = float(line.split(" for")[0][23:])
        results_dict[model_num]["Max F1, m3"] = max_f1_m3
    elif line.startswith("---"):
        model_num += 1

In [3]:
results_dict

{0: {'AUC': 3.394356338256241,
  'Time to FA': 14.6103,
  'DD': 0.6877,
  'Max F1': 0.4186,
  'COVER': 0.9169,
  'Max COVER': 0.9521479639629712,
  'Max F1, m1': 0.21212121212121213,
  'Max F1, m2': 0.25333333333333335,
  'Max F1, m3': 0.2967741935483871},
 1: {'AUC': 4.207613218471831,
  'Time to FA': 15.0,
  'DD': 0.7851,
  'Max F1': 0.3652,
  'COVER': 0.9343,
  'Max COVER': 0.952412111527441,
  'Max F1, m1': 0.19607843137254902,
  'Max F1, m2': 0.2251655629139073,
  'Max F1, m3': 0.24761904761904763},
 2: {'AUC': 3.7302895180010758,
  'Time to FA': 14.7536,
  'DD': 0.745,
  'Max F1': 0.3411,
  'COVER': 0.9206,
  'Max COVER': 0.9521136397031813,
  'Max F1, m1': 0.18571428571428572,
  'Max F1, m2': 0.20689655172413793,
  'Max F1, m3': 0.24203821656050956},
 3: {'AUC': 3.164153011304581,
  'Time to FA': 14.9828,
  'DD': 0.788,
  'Max F1': 0.3729,
  'COVER': 0.9323,
  'Max COVER': 0.9525877507163324,
  'Max F1, m1': 0.18181818181818182,
  'Max F1, m2': 0.22085889570552147,
  'Max F1, m3

In [6]:
num_models = len(results_dict) - 1

for m in metric_names:
    m_mean = np.mean([results_dict[model_num][m] for model_num in range(num_models)])
    m_std = np.std([results_dict[model_num][m] for model_num in range(num_models)])
    print(f"{m} = {np.round(m_mean, 4)} \pm {np.round(m_std, 4)}")

AUC = 4.0904 \pm 0.6171
Time to FA = 15.0426 \pm 0.24
DD = 0.7631 \pm 0.0744
Max F1 = 0.3355 \pm 0.0242
COVER = 0.9373 \pm 0.0068
Max COVER = 0.9523 \pm 0.0002
Max F1, m1 = 0.2231 \pm 0.0362
Max F1, m2 = 0.2424 \pm 0.0286
Max F1, m3 = 0.2685 \pm 0.0238
